In [ ]:

## Delete

from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Function to delete all nodes and relationships
def delete_all_nodes_and_relationships(tx):
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    tx.run(query)

# Connect to Neo4j and execute the deletion
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            session.write_transaction(delete_all_nodes_and_relationships)
            print("All nodes and relationships have been deleted.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
## Create relationships and nodes

import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# CSV file path
csv_file_path = "/content/GoodCSV (1).csv"  # Update with your actual file path

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Clean the data (optional, remove rows with NaN in required columns)
df = df.dropna(subset=["Subject", "Relationship", "Object"])

# Function to create nodes and relationships in Neo4j
def create_nodes_and_relationships(tx, subject, relationship, obj):
    # Create Cypher query
    query = """
    MERGE (subject:Node {name: $subject})
    MERGE (object:Node {name: $object})
    MERGE (subject)-[rel:%s]->(object)
    """ % relationship.upper()  # Relationship types should be in uppercase

    # Execute query
    tx.run(query, subject=subject, object=obj)

# Connect to Neo4j and process the data
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            for index, row in df.iterrows():
                subject = row["Subject"]
                relationship = row["Relationship"]
                obj = row["Object"]

                session.write_transaction(create_nodes_and_relationships, subject, relationship, obj)
                print(f"Processed row {index}: {subject} -[{relationship}]-> {obj}")

    print("Data import complete.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
## View all nodes


import sys
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

# -------------------------- Configuration -------------------------- #

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"  # Replace with your Neo4j URI
AUTH = ("nest", "neurons-newbie")      # Replace with your Neo4j username and password

# Label of the nodes to inspect (optional, adjust if necessary)
NODE_LABEL = "Node"  # Replace with your actual node label if different

# Prefix to exclude
EXCLUDE_PREFIX = "NCT"

# Maximum number of nodes to retrieve (optional)
LIMIT = 1000  # Adjust or set to None for no limit

# -------------------------------------------------------------------- #

def get_nodes_not_starting_with(driver, label, prefix, limit=None):
    """
    Retrieves names of all nodes with a specific label that do not start with a given prefix.

    Args:
        driver (GraphDatabase.driver): The Neo4j driver instance.
        label (str): The label of the nodes to filter.
        prefix (str): The prefix to exclude.
        limit (int, optional): Maximum number of nodes to retrieve. Defaults to None.

    Returns:
        list: A list of node names matching the criteria.
    """
    # Construct the Cypher query with dynamic parts
    if label:
        label_filter = f":{label}"
    else:
        label_filter = ""

    # Build the MATCH and WHERE clauses
    query = f"""
    MATCH (n{label_filter})
    WHERE NOT n.name STARTS WITH $prefix
    RETURN n.name AS NodeName
    """
    if limit:
        query += f" LIMIT {limit}"

    nodes = []
    try:
        with driver.session() as session:
            result = session.run(query, prefix=prefix)
            for record in result:
                nodes.append(record["NodeName"])
    except ServiceUnavailable as e:
        print(f"❌ ServiceUnavailable Error: {e}")
    except Exception as e:
        print(f"❌ An error occurred while retrieving nodes: {e}")
    return nodes

def display_nodes(nodes, label, prefix):
    """
    Displays the list of node names in a readable format.

    Args:
        nodes (list): List of node names.
        label (str): Label of the nodes.
        prefix (str): Prefix that was excluded.
    """
    if not nodes:
        print(f"No nodes with label '{label}' found that do not start with '{prefix}'.")
        return

    print(f"=== Nodes with label '{label}' NOT starting with '{prefix}' ===\n")
    for idx, node_name in enumerate(nodes, start=1):
        print(f"{idx}. {node_name}")
    print(f"\nTotal Nodes Found: {len(nodes)}")

def main():
    print("=== Retrieving Nodes Not Starting with 'NCT' ===\n")

    # Initialize Neo4j driver
    try:
        driver = GraphDatabase.driver(URI, auth=AUTH)
        print("✅ Successfully connected to Neo4j.\n")
    except Exception as e:
        print(f"❌ Failed to create the driver: {e}")
        sys.exit(1)

    # Retrieve nodes
    print(f"Retrieving nodes with label '{NODE_LABEL}' that do NOT start with '{EXCLUDE_PREFIX}'...\n")
    nodes = get_nodes_not_starting_with(driver, NODE_LABEL, EXCLUDE_PREFIX, LIMIT)

    # Display nodes
    display_nodes(nodes, NODE_LABEL, EXCLUDE_PREFIX)

    # Close the driver connection
    driver.close()
    print("\n🔒 Neo4j connection closed.")

if __name__ == "__main__":
    main()


In [ ]:
## Check existance


import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Function to check if a node exists in Neo4j
def check_node_exists(tx, node_name):
    # Cypher query to check if the node exists
    query = """
    MATCH (node:Node {name: $node_name})
    RETURN COUNT(node) > 0 AS node_exists
    """
    
    # Execute the query and return the result
    result = tx.run(query, node_name=node_name).single()
    return result["node_exists"]

# Connect to Neo4j and check for a node
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            # Specify the node name to check
            node_name_to_check = "'LDL cholesterol levels'"  # Replace with the actual node name

            # Check if the node exists
            node_exists = session.read_transaction(check_node_exists, node_name_to_check)

            if node_exists:
                print(f"The node '{node_name_to_check}' exists in the database.")
            else:
                print(f"The node '{node_name_to_check}' does not exist in the database.")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
## Single Iterative existance

import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Function to check if a node exists in Neo4j
def check_node_exists(tx, node_name):
    # Cypher query to check if the node exists
    query = """
    MATCH (node:Node {name: $node_name})
    RETURN COUNT(node) > 0 AS node_exists
    """
    
    # Execute the query and return the result
    result = tx.run(query, node_name=node_name).single()
    return result["node_exists"]

# Load guide.csv into a DataFrame
csv_file_path = "/content/filtered_results_with_similars (2).csv"  # Update with the correct path to your CSV file
df = pd.read_csv(csv_file_path)

# Connect to Neo4j and check nodes for the 'Similar_Objects' column in the 3rd row
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            # Get the Similar_Objects column from the 3rd row
            similar_objects = df.loc[26, "Similar_Objects"]

            # Convert the Similar_Objects string to a list
            similar_objects_list = eval(similar_objects) if similar_objects else []

            # Iterate over the list and check if each node exists
            for node_name in similar_objects_list:
                node_exists = session.read_transaction(check_node_exists, node_name)
                if node_exists:
                    print(f"The node '{node_name}' exists in the database.")
                else:
                    print(f"The node '{node_name}' does not exist in the database.")

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
## Entire Double Loop


import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Function to check if a node exists in Neo4j
def check_node_exists(tx, node_name):
    # Cypher query to check if the node exists
    query = """
    MATCH (node:Node {name: $node_name})
    RETURN COUNT(node) > 0 AS node_exists
    """
    
    # Execute the query and return the result
    result = tx.run(query, node_name=node_name).single()
    return result["node_exists"]

# Load guide.csv into a DataFrame
csv_file_path = "/content/filtered_results_with_similars (2).csv"  # Update with the correct path to your CSV file
df = pd.read_csv(csv_file_path)

# Connect to Neo4j and check nodes for the 'Similar_Objects' column for all rows
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            # Iterate over each row in the DataFrame
            for index, row in df.iterrows():
                object_name = row["Object"]
                similar_objects = row["Similar_Objects"]

                # Convert the Similar_Objects string to a list
                similar_objects_list = eval(similar_objects) if similar_objects else []

                print(f"Checking similar objects for '{object_name}':")

                # Iterate over the list and check if each node exists
                for node_name in similar_objects_list:
                    node_exists = session.read_transaction(check_node_exists, node_name)
                    if node_exists:
                        print(f"  - The similar object '{node_name}' exists in the database.")
                    else:
                        print(f"  - The similar object '{node_name}' does not exist in the database.")

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:

## Finally delete!!!!!!!!


import pandas as pd
import ast
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Function to check if a node exists in Neo4j
def check_node_exists(tx, node_name):
    query = """
    MATCH (node:Node {name: $node_name})
    RETURN COUNT(node) > 0 AS node_exists
    """
    result = tx.run(query, node_name=node_name).single()
    return result["node_exists"]

# Function to delete a similar object node and repoint its relationships
def delete_and_repoint(tx, similar_object, main_object):
    query = """
    MATCH (similar:Node {name: $similar_object})-[r]-(other)
    WITH similar, r, other
    MATCH (main:Node {name: $main_object})
    CALL {
        WITH r
        RETURN type(r) AS relationship_type
    }
    CREATE (main)-[newRel:`${relationship_type}`]->(other)
    DELETE r, similar
    """
    tx.run(query, similar_object=similar_object, main_object=main_object)

# Load guide.csv into a DataFrame
csv_file_path = "/content/filtered_results_with_similars (2).csv"  # Update with the correct path to your CSV file
df = pd.read_csv(csv_file_path)

# Connect to Neo4j and process the data
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            for index, row in df.iterrows():
                object_name = row["Object"]
                similar_objects = row["Similar_Objects"]

                # Convert Similar_Objects string to a list safely using ast.literal_eval
                similar_objects_list = ast.literal_eval(similar_objects) if similar_objects else []

                # Check if the main object exists
                main_object_exists = session.read_transaction(check_node_exists, object_name)

                if not main_object_exists:
                    print(f"The main object '{object_name}' does not exist in the database. Skipping.")
                    continue

                print(f"Processing similar objects for '{object_name}':")

                for similar_object in similar_objects_list:
                    # Check if the similar object exists
                    similar_object_exists = session.read_transaction(check_node_exists, similar_object)

                    if similar_object_exists:
                        print(f"  - The similar object '{similar_object}' exists. Deleting and repointing relationships.")
                        session.write_transaction(delete_and_repoint, similar_object, object_name)
                    else:
                        print(f"  - The similar object '{similar_object}' does not exist. Skipping.")

except Exception as e:
    print(f"An error occurred: {e}")



